<a href="https://colab.research.google.com/github/marianomirabal/etl-ibnorca/blob/main/notebooks/etl_ibnorca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl sqlalchemy requests beautifulsoup4 google
!apt update
!apt install -y chromium-chromedriver

import pandas as pd
import time
import re
import requests
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from googlesearch import search
from urllib.parse import urlparse
import random

# 1. Configuración de user-agents para evitar bloqueos
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1'
]

def get_random_user_agent():
    return random.choice(USER_AGENTS)

# 2. Función para extraer datos de Google Knowledge Panel
def extract_google_knowledge_panel(html_content):
    """Extrae información del panel de conocimiento de Google"""
    soup = BeautifulSoup(html_content, 'html.parser')
    data = {}

    # Extraer nombre de la empresa
    try:
        name_element = soup.find('div', {'data-attrid': 'title'})
        data['nombre'] = name_element.get_text() if name_element else ''
    except:
        data['nombre'] = ''

    # Extraer tipo de empresa
    try:
        type_element = soup.find('div', {'class': 'wwUB2c'})
        data['tipo'] = type_element.get_text() if type_element else ''
    except:
        data['tipo'] = ''

    # Extraer descripción
    try:
        desc_element = soup.find('div', {'class': 'kno-rdesc'})
        if desc_element:
            desc_text = desc_element.find('span').get_text()
            data['descripcion'] = desc_text
        else:
            data['descripcion'] = ''
    except:
        data['descripcion'] = ''

    # Extraer dirección
    try:
        address_element = soup.find('span', {'class': 'LrzXr'})
        data['direccion'] = address_element.get_text() if address_element else ''
    except:
        data['direccion'] = ''

    # Extraer teléfono
    try:
        phone_element = soup.find('span', {'class': 'LrzXr zdqRlf kno-fv'})
        data['telefono'] = phone_element.get_text() if phone_element else ''
    except:
        data['telefono'] = ''

    # Extraer sitio web
    try:
        web_element = soup.find('div', {'class': 'wwUB2c PZPZlf'})
        if web_element:
            web_link = web_element.find('a')
            data['sitio_web'] = web_link['href'] if web_link else ''
        else:
            data['sitio_web'] = ''
    except:
        data['sitio_web'] = ''

    return data

# 3. Función para buscar información de empresa en Google
def search_company_info(razon_social):
    """Busca información de la empresa en Google"""
    try:
        print(f"Buscando en Google: {razon_social}")

        # Buscar en Google
        query = f"{razon_social} Bolivia sitio web"
        search_results = list(search(query, num=5, stop=5, pause=2))

        # Seleccionar el primer resultado que parezca oficial
        company_url = None
        for url in search_results:
            parsed_url = urlparse(url)
            domain = parsed_url.netloc.lower()

            # Filtrar sitios que no son relevantes
            if any(exclude in domain for exclude in ['google', 'facebook', 'linkedin', 'twitter', 'youtube', 'wikipedia']):
                continue

            # Preferir sitios que contengan el nombre de la empresa
            if razon_social.lower().split()[0] in domain:
                company_url = url
                break

        if not company_url:
            company_url = search_results[0] if search_results else None

        if not company_url:
            print("No se encontraron resultados en Google")
            return None

        print(f"URL encontrada: {company_url}")

        # Obtener contenido HTML
        headers = {'User-Agent': get_random_user_agent()}
        response = requests.get(company_url, headers=headers, timeout=10)

        if response.status_code != 200:
            print(f"Error al acceder al sitio: {response.status_code}")
            return None

        # Extraer información de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Intentar extraer información estructurada
        company_data = {
            'sitio_web': company_url,
            'direccion': '',
            'ciudad': '',
            'telefono': '',
            'descripcion': ''
        }

        # Extraer dirección
        address_patterns = [
            r'direcci[oó]n', r'dir\.', r'ubicaci[oó]n',
            r'address', r'location', r'contacto'
        ]

        for pattern in address_patterns:
            element = soup.find(string=re.compile(pattern, re.I))
            if element:
                parent = element.find_parent()
                # Buscar el texto de dirección en elementos cercanos
                for sibling in parent.find_next_siblings():
                    text = sibling.get_text().strip()
                    if text and len(text) > 10 and not any(tag in text.lower() for tag in ['email', 'teléfono', 'phone']):
                        company_data['direccion'] = text
                        break
                if company_data['direccion']:
                    break

        # Extraer teléfono
        phone_patterns = [
            r'tel[ée]fono', r'phone', r'contacto',
            r'call', r'telephone', r'celular'
        ]

        for pattern in phone_patterns:
            element = soup.find(string=re.compile(pattern, re.I))
            if element:
                parent = element.find_parent()
                # Buscar números de teléfono en elementos cercanos
                for sibling in parent.find_next_siblings():
                    text = sibling.get_text().strip()
                    # Buscar números de teléfono en el texto
                    phones = re.findall(r'\(?\d{3,4}\)?[\s-]?\d{3,4}[\s-]?\d{3,4}', text)
                    if phones:
                        company_data['telefono'] = ', '.join(phones)
                        break
                if company_data['telefono']:
                    break

        # Extraer descripción (meta description o primer párrafo)
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        if meta_desc and meta_desc.get('content'):
            company_data['descripcion'] = meta_desc['content']
        else:
            # Buscar el primer párrafo significativo
            for p in soup.find_all('p'):
                text = p.get_text().strip()
                if len(text) > 100 and not any(word in text.lower() for word in ['copyright', 'derechos', 'reservados']):
                    company_data['descripcion'] = text
                    break

        # Extraer ciudad de la dirección o descripción
        ciudades_bolivia = [
            'La Paz', 'Cochabamba', 'Santa Cruz', 'Oruro', 'Potosí',
            'Tarija', 'Sucre', 'Trinidad', 'Cobija', 'El Alto'
        ]

        for ciudad in ciudades_bolivia:
            if ciudad in company_data['direccion']:
                company_data['ciudad'] = ciudad
                break
            elif ciudad in company_data['descripcion']:
                company_data['ciudad'] = ciudad
                break

        print("Datos obtenidos con éxito")
        return company_data

    except Exception as e:
        print(f"Error en búsqueda de Google: {str(e)}")
        return None

# 4. Mapeo de actividad a sector IBNORCA
def map_actividad_to_sector(descripcion):
    if not descripcion:
        return "SERVICIOS"

    descripcion = descripcion.lower()

    mapping = {
        r'agro|agric|pecuar|aliment': 'AGROPECUARIO, AGRÍCOLA Y ALIMENTOS',
        r'construc|obra civil|edificac|cemento|hormigón': 'CONSTRUCCIÓN',
        r'energ|eléctr|solar|eólica': 'ENERGÍA',
        r'hidrocarb|petrol|gas natural': 'HIDROCARBUROS',
        r'industr|manufactur|fabrica': 'INDUSTRIA',
        r'mecánic|automotr|vehículo': 'MECÁNICA',
        r'medio ambien|ecolog|reciclaj': 'MEDIO AMBIENTE Y AGUA',
        r'miner|minera|extracción': 'MINERÍA',
        r'salud|médic|hospital|farmac': 'SALUD',
        r'segurid|riesgo|protección': 'SEGURIDAD OCUPACIONAL',
        r'tecnolog|software|informática|tic': 'TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN',
        r'transport|logístic|flet': 'TRANSPORTE',
        r'norma|certifica|calidad|iso': 'INFRAESTRUCTURA DE LA CALIDAD',
        r'comerc|venta|distribución': 'COMERCIO',
        r'financ|banc|seguro|inversión': 'FINANZAS'
    }

    for pattern, sector in mapping.items():
        if re.search(pattern, descripcion):
            return sector

    return "SERVICIOS"

# 5. Determinar tamaño de empresa basado en descripción
def determinar_tamano_empresa(descripcion):
    if not descripcion:
        return "Mediana"

    descripcion = descripcion.lower()

    size_keywords = {
        "Grande": [r'corporaci[oó]n', r'holding', r'grupo', r'multinacional', r'subsidiaria', r'filial'],
        "Mediana": [r'empresa establecida', r'mediano porte', r'mediano tamaño', r'equipo profesional'],
        "Pequeña": [r'pyme', r'pequeña empresa', r'emprendimiento', r'equipo reducido'],
        "Micro": [r'microempresa', r'emprendedor', r'unipersonal', r'autónomo']
    }

    for size, patterns in size_keywords.items():
        for pattern in patterns:
            if re.search(pattern, descripcion):
                return size

    return "Mediana"

# 6. Función principal ETL
def main_etl():
    # Cargar datos
    print("Cargando archivo Excel...")
    df = pd.read_excel('/content/Clientes_faltantes.xlsx')

    # 1. Limpieza de nombres de columnas
    df.columns = df.columns.str.strip().str.replace(r'\.+','_',regex=True).str.replace(r'\s+','_',regex=True)

    # 2. Clasificar Tipo de Persona
    print("Clasificando tipo de persona...")
    keywords = ['SRL','S.R.L.','S.R.L','SRL.','S R L.','R.L','RL.','S.A','S.A.','SA.','LTDA','LTDA.',
                'CORPORACIÓN','COOPERATIVA','GRUPO','SOCIEDAD DE RESPONSABILIDAD LIMITADA','LIMITADA',
                'ASOCIACION','ASOCIACIÓN','BANCO','CLUB','EBIH','EMAPSA']
    df['Tipo_de_persona'] = df['Column1_razon_social'].apply(
        lambda x: 'Empresa' if any(k in str(x).upper() for k in keywords) else 'Persona'
    )

    # 3. Determinar Tipo de Empresa
    print("Determinando tipo de empresa...")
    empresas_publicas = ["bolsa boliviana de valores", "ebih", "empresa boliviana de industrialización de hidrocarburos",
        "yacimientos petrolíferos fiscales bolivianos", "ypfb", "mi teleférico", "enabol", "tam", "esabol",
        "emapa", "ebc", "cofadena", "quipus", "mutún", "easba", "yacana", "editorial del estado", "eba", "abe",
        "bolivia tv", "gestora", "entel", "datacom", "entel dd", "entel bolivia sac", "comibol", "ebo", "emk",
        "emv", "emc", "emcorocoro", "emh", "ende", "ende corporación", "ende servicios y construcciones",
        "ende tecnologías", "ende delbeni", "delapaz", "elfec", "ende corani", "ende andina", "ende deoruro",
        "ende guaracachi", "ende transmisión", "ende valle hermoso", "rio eléctrico", "ypfb corporación",
        "ypfb logística", "ypfb transierra", "ypfb transporte", "air bp bolivia", "ypfb refinación",
        "ypfb andina", "tab", "ypfb chaco", "gtb", "chaco energías", "ylb", "eepaf", "eeps", "cartonbol",
        "papelbol", "ecebol", "boa", "envibol", "etasa", "kokabol", "ibae", "b-agro", "asp-b", "dab"]

    def determinar_tipo_empresa(nombre):
        nombre = str(nombre).lower()
        if any(publica in nombre for publica in empresas_publicas):
            return "Pública"
        elif any(palabra in nombre for palabra in ["universidad", "colegio", "instituto", "escuela superior", "tecnológico"]):
            return "Académica"
        elif any(palabra in nombre for palabra in ["ong", "organización no gubernamental", "fundación", "asociación civil", "asociación sin fines"]):
            return "ONG"
        else:
            return "Privada"

    df.loc[df["Tipo_de_persona"] == "Empresa", "Tipo_de_Empresa"] = df.loc[
        df["Tipo_de_persona"] == "Empresa", "Column1_razon_social"
    ].apply(determinar_tipo_empresa)

    # 4. Adivinar Sector Empresarial
    print("Asignando sector empresarial preliminar...")
    def adivinar_sector(nombre):
        nombre = str(nombre).lower()
        if any(palabra in nombre for palabra in ["hospital", "clínica", "med", "salud", "bio", "farma", "laboratorio","laboratorios","lab","pharma"]):
            return "SALUD"
        elif any(palabra in nombre for palabra in ["seguridad", "safety", "riesgo", "osha"]):
            return "SEGURIDAD OCUPACIONAL"
        elif any(palabra in nombre for palabra in ["minera", "mining"]):
            return "MINERÍA"
        elif any(palabra in nombre for palabra in ["ingeniería", "mecánica", "industrial"]):
            return "MECÁNICA"
        elif any(palabra in nombre for palabra in ["construct", "construcciones", "obras", "infra", "arquitect","soboce","cemento","hormigón"]):
            return "CONSTRUCCIÓN"
        elif any(palabra in nombre for palabra in ["agro", "alimentos", "granja", "avícola", "leche", "lácteo", "cárnico","imba","kral","cereal","cerveza","cervecería","food"]):
            return "AGROPECUARIO, AGRÍCOLA Y ALIMENTOS"
        elif any(palabra in nombre for palabra in ["gas", "petrol", "glp", "combustible","hidrocarburos","surtidor"]):
            return "HIDROCARBUROS"
        elif any(palabra in nombre for palabra in ["energía", "eléctrica", "solar", "hidroeléctrica","electrificacion","electrificación","energy"]):
            return "ENERGÍA"
        elif any(palabra in nombre for palabra in ["agua", "medio ambiente", "ambiental", "ecología"]):
            return "MEDIO AMBIENTE Y AGUA"
        elif any(palabra in nombre for palabra in ["transporte", "logística", "shipping", "envío"]):
            return "TRANSPORTE"
        elif any(palabra in nombre for palabra in ["sistemas", "software", "informática", "tecnología","tech","techno"]):
            return "TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN"
        elif any(palabra in nombre for palabra in ["norma", "certifica", "ibnorca", "calidad", "iso"]):
            return "INFRAESTRUCTURA DE LA CALIDAD"
        elif any(palabra in nombre for palabra in ["industria", "fábrica", "producción"]):
            return "INDUSTRIA"
        elif any(palabra in nombre for palabra in ["hogar", "doméstico", "electrodoméstico", "mueble"]):
            return "EQUIPAMIENTO DOMÉSTICO Y COMERCIAL"
        else:
            return "SERVICIOS"

    df.loc[df["Tipo_de_persona"] == "Empresa", "Sector_Empresarial"] = df.loc[
        df["Tipo_de_persona"] == "Empresa", "Column1_razon_social"
    ].apply(adivinar_sector)

    # 5. Enriquecimiento con Google (solo para empresas)
    print("\nIniciando enriquecimiento con datos de Google...")
    empresas_idx = df[df["Tipo_de_persona"] == "Empresa"].index

    for i, idx in enumerate(empresas_idx):
        razon_social = df.loc[idx, 'Column1_razon_social']
        print(f"\n[{i+1}/{len(empresas_idx)}] Procesando empresa: {razon_social}")

        # Obtener datos de Google
        google_data = search_company_info(razon_social)

        if google_data:
            # Actualizar dirección y ciudad
            df.at[idx, 'Dirección'] = google_data.get('direccion', '')
            df.at[idx, 'Ciudad'] = google_data.get('ciudad', '')

            # Actualizar sector basado en descripción
            descripcion = google_data.get('descripcion', '')
            if descripcion:
                sector_google = map_actividad_to_sector(descripcion)
                # Solo actualizar si es diferente del valor por defecto
                if sector_google != "SERVICIOS":
                    df.at[idx, 'Sector_Empresarial'] = sector_google

            # Determinar tamaño basado en descripción
            df.at[idx, 'Tamaño_de_la_Empresa'] = determinar_tamano_empresa(descripcion)

            df.at[idx, 'Fuente_Datos'] = 'Google'
            df.at[idx, 'Sitio_Web'] = google_data.get('sitio_web', '')
            df.at[idx, 'Teléfono'] = google_data.get('telefono', '')
        else:
            df.at[idx, 'Fuente_Datos'] = 'No encontrado'

        # Pausa para evitar bloqueos
        sleep_time = random.randint(5, 15)
        print(f"Pausa de {sleep_time} segundos...")
        time.sleep(sleep_time)

    # 6. País (Bolivia por defecto para empresas)
    df.loc[df["Tipo_de_persona"] == "Empresa", "País"] = "Bolivia"

    # 7. Guardar resultados
    print("\nGuardando resultados en Excel...")
    df.to_excel("Clientes_IBNORCA_Completos.xlsx", index=False)

    # 8. Cargar en SQLite (opcional)
    print("Guardando en base de datos SQLite...")
    engine = create_engine('sqlite:///clientes_ibnorca.db')
    df.to_sql('clientes_limpios', engine, if_exists='replace', index=False)

    return df

# Ejecutar el proceso ETL
print("Iniciando proceso ETL completo...")
df_final = main_etl()
print("\nProceso ETL completado exitosamente!")
print(f"Total de registros procesados: {len(df_final)}")
print("Archivo guardado: Clientes_IBNORCA_Completos.xlsx")

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
37 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR